<a href="https://colab.research.google.com/github/yadavrishikesh/BayesNF/blob/main/Portland_DailyTraffic_DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary packages
!pip install -q bayesnf cartopy contextily geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00


In [2]:
import warnings
warnings.simplefilter('ignore')

import contextily as ctx
import geopandas as gpd
import jax
import numpy as np
import pandas as pd
import os
import datetime
from urllib.parse import urlparse
from bayesnf.spatiotemporal import BayesianNeuralFieldMAP

In [3]:
def run_portland_model(train_url, test_url):
    """
    Trains a Bayesian Neural Field MAP model on specified Portland train data and tests on test data.

    Parameters:
    - train_url (str): URL to the training dataset CSV file.
    - test_url (str): URL to the test dataset CSV file.

    Returns:
    - predictions_df (DataFrame): DataFrame containing true values, predicted mean, and quantiles.
    """
    # Determine base directory from train_url and create unique outputs directory
    train_name = os.path.basename(urlparse(train_url).path).split('.')[0]
    test_name = os.path.basename(urlparse(test_url).path).split('.')[0]
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(os.getcwd(), 'outputs', f'{train_name}_{test_name}_{timestamp}')
    os.makedirs(output_dir, exist_ok=True)

    # Download and load datasets
    !wget -q {train_url} -O train_data.csv
    !wget -q {test_url} -O test_data.csv
    df_train = pd.read_csv('train_data.csv', index_col=0, parse_dates=['datetime'])
    df_test = pd.read_csv('test_data.csv', index_col=0, parse_dates=['datetime'])

    # Log-transform response in training data
    df_train_log = df_train.copy()
    df_train_log['response'] = np.log1p(df_train_log['response'])

    # Define and train the Bayesian Neural Field model
    model = BayesianNeuralFieldMAP(
        width=512,
        depth=2,
        freq='D',
        seasonality_periods=['W'],
        num_seasonal_harmonics=[2],
        feature_cols=['datetime', 'lat', 'lon', 'speed', 'occupancy'],
        target_col='response',
        observation_model='NORMAL',
        timetype='index',
        standardize=['lat', 'lon', 'speed', 'occupancy'],
    )

    model = model.fit(df_train_log,
                      seed=jax.random.PRNGKey(0),
                      ensemble_size=10,
                      num_epochs=1000)

    # Process test data
    df_test_cleaned = df_test.dropna(subset=['response'])
    df_test_cleaned_log = df_test_cleaned.copy()
    df_test_cleaned_log['response'] = np.log1p(df_test_cleaned_log['response'])

    # Make predictions
    yhat, yhat_quantiles = model.predict(df_test_cleaned, quantiles=(0.025, 0.5, 0.975))

    # Compile results into DataFrame
    predictions_df = pd.DataFrame({
        'datetime': df_test_cleaned.index,
        'True_Values': df_test_cleaned['response'],
        'Predicted_Mean': yhat_quantiles[1],
        'Quantile_0.025': yhat_quantiles[0],
        'Quantile_0.975': yhat_quantiles[2]
    })

    # Save output in the unique outputs directory
    output_path = os.path.join(output_dir, 'predictions_output.csv')
    predictions_df.to_csv(output_path, index=False)

    return predictions_df

In [4]:
# Spatial predictions
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_daily_pred-type_spatIntpl.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_daily_pred-type_spatIntpl.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,10.067645,1.766386,1.243296,2.206590
2,2,10.596935,1.464419,1.196171,1.903005
3,3,10.639263,1.939993,1.520911,2.289216
4,4,10.653771,1.858743,1.447320,2.118954
5,5,10.687732,1.677076,1.071427,2.130167


In [ ]:
# Forecasting
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_daily_pred-type_forecast.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_daily_pred-type_forecast.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,10.665578,2.412498,2.193699,2.599449
2,2,10.610340,2.383557,2.163939,2.569709
3,3,10.500537,2.370196,2.147752,2.555152
4,4,10.430196,2.359263,2.133215,2.541827
5,5,10.603337,2.418967,2.200211,2.603979


In [ ]:
# Space-time predictions
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_daily_pred-type_spaceTime.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_daily_pred-type_spaceTime.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,10.707550,2.218443,2.025122,2.427753
2,2,10.545157,2.326962,2.069804,2.514436
3,3,10.419510,2.330055,1.979126,2.530117
4,4,10.402382,2.461663,2.165895,2.673280
5,5,10.573775,2.365219,2.153480,2.615012


In [ ]:
# Mount Google Drive to access it from Colab
from google.colab import drive
drive.mount('/content/drive')

# Copy the 'outputs' directory to Google Drive
!cp -r /content/outputs /content/drive/MyDrive/BayesNF/outputs_Portland_daily


Mounted at /content/drive
